In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests

In [2]:
# load in csvs and store in dataframe
honey_path = "bee_data/6honey.csv"
colony_path = "bee_data/bee_colonies_state.csv"

honey_df = pd.read_csv(honey_path)
colony_df = pd.read_csv(colony_path)

honey_df.head()
#colony_df.head()

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2019,MID DEC,NaN,NATIONAL,US TOTAL,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"41,022,000",NaN
1,SURVEY,2019,MID DEC,NaN,STATE,ALABAMA,1.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"44,000",NaN
2,SURVEY,2019,MID DEC,NaN,STATE,ARIZONA,4.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"201,000",NaN
3,SURVEY,2019,MID DEC,NaN,STATE,ARKANSAS,5.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"176,000",NaN
4,SURVEY,2019,MID DEC,NaN,STATE,CALIFORNIA,6.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"3,216,000",NaN


HONEY

In [3]:
honey_cleanup = honey_df[['Year', 'Period', 'State','Commodity','Data Item','Domain','Domain Category','Value']]
honey_cleanup

,Year,Period,State,Commodity,Data Item,Domain,Domain Category,Value
0,2019,MID DEC,US TOTAL,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"41,022,000"
1,2019,MID DEC,ALABAMA,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"44,000"
2,2019,MID DEC,ARIZONA,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"201,000"
3,2019,MID DEC,ARKANSAS,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"176,000"
4,2019,MID DEC,CALIFORNIA,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"3,216,000"
...,...,...,...,...,...,...,...,...
1465,1987,MID DEC,VIRGINIA,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"120,000"
1466,1987,MID DEC,WASHINGTON,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"206,000"
1467,1987,MID DEC,WEST VIRGINIA,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"207,000"
1468,1987,MID DEC,WISCONSIN,HONEY,"HONEY - STOCKS, MEASURED IN LB",TOTAL,NOT SPECIFIED,"1,160,000"


In [4]:
clean_honey = honey_cleanup[['Year','State','Value']].copy()
clean_honey

,Year,State,Value
0,2019,US TOTAL,"41,022,000"
1,2019,ALABAMA,"44,000"
2,2019,ARIZONA,"201,000"
3,2019,ARKANSAS,"176,000"
4,2019,CALIFORNIA,"3,216,000"
...,...,...,...
1465,1987,VIRGINIA,"120,000"
1466,1987,WASHINGTON,"206,000"
1467,1987,WEST VIRGINIA,"207,000"
1468,1987,WISCONSIN,"1,160,000"


In [5]:
clean_honey['Value'] = clean_honey['Value'].str.replace(',','')

In [6]:
clean_honey['Value'] = clean_honey['Value'].astype(int)
clean_honey.head()

,Year,State,Value
0,2019,US TOTAL,41022000
1,2019,ALABAMA,44000
2,2019,ARIZONA,201000
3,2019,ARKANSAS,176000
4,2019,CALIFORNIA,3216000


In [7]:
cleaned_honey = clean_honey.rename(columns={'Year':'year','State':'state','Value':'lbs_of_honey'})
cleaned_honey

,year,state,lbs_of_honey
0,2019,US TOTAL,41022000
1,2019,ALABAMA,44000
2,2019,ARIZONA,201000
3,2019,ARKANSAS,176000
4,2019,CALIFORNIA,3216000
...,...,...,...
1465,1987,VIRGINIA,120000
1466,1987,WASHINGTON,206000
1467,1987,WEST VIRGINIA,207000
1468,1987,WISCONSIN,1160000


Colonies

In [8]:
colony_cleanup = colony_df[['Year', 'Period', 'State','Commodity','Data Item','Domain','Domain Category','Value']].copy()
colony_cleanup

,Year,Period,State,Commodity,Data Item,Domain,Domain Category,Value
0,2020,FIRST OF JAN,US TOTAL,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"2,876,100"
1,2020,FIRST OF JAN,ALABAMA,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"5,500"
2,2020,FIRST OF JAN,ARIZONA,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"29,000"
3,2020,FIRST OF JAN,ARKANSAS,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"16,500"
4,2020,FIRST OF JAN,CALIFORNIA,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"1,350,000"
...,...,...,...,...,...,...,...,...
2464,1987,MARKETING YEAR,VIRGINIA,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"25,000"
2465,1987,MARKETING YEAR,WASHINGTON,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"75,000"
2466,1987,MARKETING YEAR,WEST VIRGINIA,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"21,000"
2467,1987,MARKETING YEAR,WISCONSIN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"92,000"


In [9]:
colony_cleanup['Value'] = colony_cleanup['Value'].str.replace(',','')
colony_cleanup

,Year,Period,State,Commodity,Data Item,Domain,Domain Category,Value
0,2020,FIRST OF JAN,US TOTAL,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,2876100
1,2020,FIRST OF JAN,ALABAMA,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,5500
2,2020,FIRST OF JAN,ARIZONA,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,29000
3,2020,FIRST OF JAN,ARKANSAS,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,16500
4,2020,FIRST OF JAN,CALIFORNIA,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,1350000
...,...,...,...,...,...,...,...,...
2464,1987,MARKETING YEAR,VIRGINIA,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,25000
2465,1987,MARKETING YEAR,WASHINGTON,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,75000
2466,1987,MARKETING YEAR,WEST VIRGINIA,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,21000
2467,1987,MARKETING YEAR,WISCONSIN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,92000


In [10]:
colony_cleanup['Value'] = colony_cleanup['Value'].astype(int)

In [11]:
## for the machine learning part we are just going with 2015 data. Taking the average of the quarters and dropping marketing year. Might set marketing year as its own table... thoughts?
colony_cleanup.drop(colony_cleanup.loc[colony_cleanup['Period'] == 'MARKETING YEAR'].index, inplace=True)

In [12]:
colony_state_year = colony_cleanup.groupby(['Year','State'], as_index = False).mean(['Value'])
colony_state_year

,Year,State,Value
0,2015,ALABAMA,7750.0
1,2015,ARIZONA,36000.0
2,2015,ARKANSAS,18125.0
3,2015,CALIFORNIA,990000.0
4,2015,COLORADO,17125.0
...,...,...,...
277,2020,VIRGINIA,6750.0
278,2020,WASHINGTON,49500.0
279,2020,WEST VIRGINIA,6750.0
280,2020,WISCONSIN,21500.0


In [13]:
cleaned_colony = colony_state_year.rename(columns={'Year':'year','State':'state','Value':'count_colonies'})
cleaned_colony

,year,state,count_colonies
0,2015,ALABAMA,7750.0
1,2015,ARIZONA,36000.0
2,2015,ARKANSAS,18125.0
3,2015,CALIFORNIA,990000.0
4,2015,COLORADO,17125.0
...,...,...,...
277,2020,VIRGINIA,6750.0
278,2020,WASHINGTON,49500.0
279,2020,WEST VIRGINIA,6750.0
280,2020,WISCONSIN,21500.0


MARKETING YEAR

In [14]:
## just in case we wanted this for graphs
marketing_year = colony_df.copy()
marketing_year.head()

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,SURVEY,2020,FIRST OF JAN,NaN,NATIONAL,US TOTAL,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"2,876,100",NaN
1,SURVEY,2020,FIRST OF JAN,NaN,STATE,ALABAMA,1.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"5,500",NaN
2,SURVEY,2020,FIRST OF JAN,NaN,STATE,ARIZONA,4.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"29,000",NaN
3,SURVEY,2020,FIRST OF JAN,NaN,STATE,ARKANSAS,5.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"16,500",NaN
4,SURVEY,2020,FIRST OF JAN,NaN,STATE,CALIFORNIA,6.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"1,350,000",NaN


In [15]:
marketing_year.drop(marketing_year.loc[marketing_year['Period'] != 'MARKETING YEAR'].index, inplace=True)
marketing_year.head()

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
94,SURVEY,2019,MARKETING YEAR,NaN,NATIONAL,US TOTAL,NaN,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"2,812,000",NaN
95,SURVEY,2019,MARKETING YEAR,NaN,STATE,ALABAMA,1.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"7,000",NaN
96,SURVEY,2019,MARKETING YEAR,NaN,STATE,ARIZONA,4.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"23,000",NaN
97,SURVEY,2019,MARKETING YEAR,NaN,STATE,ARKANSAS,5.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"20,000",NaN
98,SURVEY,2019,MARKETING YEAR,NaN,STATE,CALIFORNIA,6.0,NaN,NaN,NaN,...,NaN,NaN,0,NaN,HONEY,"HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...",TOTAL,NOT SPECIFIED,"335,000",NaN


In [16]:
marketing_cleanup = marketing_year[['Year','State','Value']].copy()
marketing_cleanup.head()

,Year,State,Value
94,2019,US TOTAL,"2,812,000"
95,2019,ALABAMA,"7,000"
96,2019,ARIZONA,"23,000"
97,2019,ARKANSAS,"20,000"
98,2019,CALIFORNIA,"335,000"


In [17]:
marketing_cleanup['Value'] = marketing_cleanup['Value'].str.replace(',','')
marketing_cleanup

,Year,State,Value
94,2019,US TOTAL,2812000
95,2019,ALABAMA,7000
96,2019,ARIZONA,23000
97,2019,ARKANSAS,20000
98,2019,CALIFORNIA,335000
...,...,...,...
2464,1987,VIRGINIA,25000
2465,1987,WASHINGTON,75000
2466,1987,WEST VIRGINIA,21000
2467,1987,WISCONSIN,92000


In [18]:
marketing_cleanup['Value'] = marketing_cleanup['Value'].astype(int)

In [19]:
cleaned_marketing_year = marketing_cleanup.rename(columns={'Year':'year','State':'state','Value':'marketing_year_val'})
cleaned_marketing_year

,year,state,marketing_year_val
94,2019,US TOTAL,2812000
95,2019,ALABAMA,7000
96,2019,ARIZONA,23000
97,2019,ARKANSAS,20000
98,2019,CALIFORNIA,335000
...,...,...,...
2464,1987,VIRGINIA,25000
2465,1987,WASHINGTON,75000
2466,1987,WEST VIRGINIA,21000
2467,1987,WISCONSIN,92000


DEADOUT COLLAPSE AND PESTICIDES

In [20]:
deadout_df = pd.read_csv("bee_data/bee_loss_deadout.csv")
collapse_df = pd.read_csv("bee_data/colony_collapse_bees.csv")
pesticides_df = pd.read_csv("bee_data/pesticides_death_bees.csv")

deadout_df.dropna(axis=1, how="all").copy()
collapse_df.dropna(axis=1, how="all").copy()
pesticides_df.dropna(axis=1, how="all").copy()

,Program,Year,Period,Geo Level,State,State ANSI,watershed_code,Commodity,Data Item,Domain,Domain Category,Value
0,SURVEY,2020,JAN THRU MAR,NATIONAL,US TOTAL,NaN,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,5
1,SURVEY,2020,JAN THRU MAR,STATE,ALABAMA,1.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,0.1
2,SURVEY,2020,JAN THRU MAR,STATE,ARIZONA,4.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,0
3,SURVEY,2020,JAN THRU MAR,STATE,ARKANSAS,5.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,2.6
4,SURVEY,2020,JAN THRU MAR,STATE,CALIFORNIA,6.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...
985,SURVEY,2015,OCT THRU DEC,STATE,VIRGINIA,51.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,0
986,SURVEY,2015,OCT THRU DEC,STATE,WASHINGTON,53.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,0.3
987,SURVEY,2015,OCT THRU DEC,STATE,WEST VIRGINIA,54.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,3
988,SURVEY,2015,OCT THRU DEC,STATE,WISCONSIN,55.0,0,HONEY,"HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - ...",TOTAL,NOT SPECIFIED,21.9


In [21]:
deadout_df["Value"] = deadout_df["Value"].str.replace(",","")
deadout_df['Value'] = deadout_df['Value'].astype('int64')
deadout = deadout_df.groupby(['Year', 'State',], as_index=False)[['Value']].sum()
deadout = deadout.rename(columns = {
    'Year': 'year',
    'State': 'state',
    'Value': 'deadout'
    })
deadout

,year,state,deadout
0,2015,ALABAMA,4670
1,2015,ARIZONA,28100
2,2015,ARKANSAS,14800
3,2015,CALIFORNIA,584000
4,2015,COLORADO,10400
...,...,...,...
277,2020,VIRGINIA,2300
278,2020,WASHINGTON,4600
279,2020,WEST VIRGINIA,1670
280,2020,WISCONSIN,3200


In [22]:
collapse_df["Value"] = collapse_df["Value"].str.replace(",","")
collapse_df['Value'] = collapse_df['Value'].astype('int64')
collapse = collapse_df.groupby(['Year', 'State',], as_index=False)[['Value']].sum()
collapse = collapse.rename(columns = {
    'Year': 'year',
    'State': 'state',
    'Value': 'cc_syn'
    })
collapse

,year,state,cc_syn
0,2015,US TOTAL,332010
1,2016,US TOTAL,391130
2,2017,US TOTAL,251130
3,2018,US TOTAL,330680
4,2019,ALABAMA,430
...,...,...,...
92,2020,VIRGINIA,540
93,2020,WASHINGTON,6870
94,2020,WEST VIRGINIA,330
95,2020,WISCONSIN,1690


In [23]:
pesticides_df.drop(pesticides_df.loc[pesticides_df['Value'] == ' (Z)'].index, inplace=True)
pesticides_df['Value'] = pesticides_df['Value'].astype('float64')
pesticides_df = pesticides_df.loc[pesticides_df['Domain'] == 'TOTAL']
pesticides = pesticides_df.groupby(['Year', 'State',], as_index=False)[['Value']].sum()
pesticides = pesticides.rename(columns = {
    'Year': 'year',
    'State': 'state',
    'Value': 'pesticides'
    })
pesticides

,year,state,pesticides
0,2015,ALABAMA,7.3
1,2015,ARIZONA,40.9
2,2015,ARKANSAS,39.2
3,2015,CALIFORNIA,61.8
4,2015,COLORADO,39.6
...,...,...,...
277,2020,VIRGINIA,0.7
278,2020,WASHINGTON,1.5
279,2020,WEST VIRGINIA,1.1
280,2020,WISCONSIN,25.1


In [24]:
combined = deadout.merge(collapse, on=['state','year'])
decline_final = combined.merge(pesticides, on=['state','year'])
decline_final

,year,state,deadout,cc_syn,pesticides
0,2015,US TOTAL,1756860,332010,48.7
1,2016,US TOTAL,1674960,391130,47.8
2,2017,US TOTAL,1534610,251130,47.1
3,2018,US TOTAL,1615150,330680,53.1
4,2019,ALABAMA,2300,430,5.4
...,...,...,...,...,...
92,2020,VIRGINIA,2300,540,0.7
93,2020,WASHINGTON,4600,6870,1.5
94,2020,WEST VIRGINIA,1670,330,1.1
95,2020,WISCONSIN,3200,1690,25.1


TEMPERATURE DATA

In [25]:
temp_data = pd.read_csv("../bee_data/city_temperature.csv", low_memory =False)
temp_data

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9
...,...,...,...,...,...,...,...,...
2906322,North America,US,Additional Territories,San Juan Puerto Rico,7,27,2013,82.4
2906323,North America,US,Additional Territories,San Juan Puerto Rico,7,28,2013,81.6
2906324,North America,US,Additional Territories,San Juan Puerto Rico,7,29,2013,84.2
2906325,North America,US,Additional Territories,San Juan Puerto Rico,7,30,2013,83.8


In [26]:
temp_data.drop(temp_data.loc[temp_data['Country'] != 'US'].index, inplace=True)
temp_data

,Region,Country,State,City,Month,Day,Year,AvgTemperature
1450990,North America,US,Alabama,Birmingham,1,1,1995,50.7
1450991,North America,US,Alabama,Birmingham,1,2,1995,37.2
1450992,North America,US,Alabama,Birmingham,1,3,1995,33.2
1450993,North America,US,Alabama,Birmingham,1,4,1995,33.3
1450994,North America,US,Alabama,Birmingham,1,5,1995,26.4
...,...,...,...,...,...,...,...,...
2906322,North America,US,Additional Territories,San Juan Puerto Rico,7,27,2013,82.4
2906323,North America,US,Additional Territories,San Juan Puerto Rico,7,28,2013,81.6
2906324,North America,US,Additional Territories,San Juan Puerto Rico,7,29,2013,84.2
2906325,North America,US,Additional Territories,San Juan Puerto Rico,7,30,2013,83.8


In [27]:
temp_data.drop(temp_data.loc[temp_data['AvgTemperature'] == -99.00].index, inplace=True)
temp_data

,Region,Country,State,City,Month,Day,Year,AvgTemperature
1450990,North America,US,Alabama,Birmingham,1,1,1995,50.7
1450991,North America,US,Alabama,Birmingham,1,2,1995,37.2
1450992,North America,US,Alabama,Birmingham,1,3,1995,33.2
1450993,North America,US,Alabama,Birmingham,1,4,1995,33.3
1450994,North America,US,Alabama,Birmingham,1,5,1995,26.4
...,...,...,...,...,...,...,...,...
2906322,North America,US,Additional Territories,San Juan Puerto Rico,7,27,2013,82.4
2906323,North America,US,Additional Territories,San Juan Puerto Rico,7,28,2013,81.6
2906324,North America,US,Additional Territories,San Juan Puerto Rico,7,29,2013,84.2
2906325,North America,US,Additional Territories,San Juan Puerto Rico,7,30,2013,83.8


In [28]:
state_avg_temp = temp_data.groupby(['State','Month','Day','Year'], as_index = False).mean(['AvgTemperature'])
state_avg_temp

,State,Month,Day,Year,AvgTemperature
0,Additional Territories,1,1,1995,77.40
1,Additional Territories,1,1,1996,78.90
2,Additional Territories,1,1,1997,76.70
3,Additional Territories,1,1,1998,78.20
4,Additional Territories,1,1,1999,77.00
...,...,...,...,...,...
474257,Wyoming,12,31,2015,11.00
474258,Wyoming,12,31,2016,26.00
474259,Wyoming,12,31,2017,17.15
474260,Wyoming,12,31,2018,15.25


In [29]:
state_avg_temp_filtered = state_avg_temp[(state_avg_temp['AvgTemperature']<= 41) | (state_avg_temp['AvgTemperature']>= 95)]

In [30]:
state_count = state_avg_temp_filtered.groupby(['State','Year'], as_index = False)['AvgTemperature'].count()
state_count

,State,Year,AvgTemperature
0,Alabama,1995,35
1,Alabama,1996,34
2,Alabama,1997,27
3,Alabama,1998,15
4,Alabama,1999,15
...,...,...,...
1243,Wyoming,2016,135
1244,Wyoming,2017,130
1245,Wyoming,2018,157
1246,Wyoming,2019,172


In [31]:
cleaned_state_ext_temp = state_count.rename(columns={'Year':'year','State':'state','AvgTemperature':'extreme_temp_days'})
cleaned_state_ext_temp

,state,year,extreme_temp_days
0,Alabama,1995,35
1,Alabama,1996,34
2,Alabama,1997,27
3,Alabama,1998,15
4,Alabama,1999,15
...,...,...,...
1243,Wyoming,2016,135
1244,Wyoming,2017,130
1245,Wyoming,2018,157
1246,Wyoming,2019,172


In [32]:
cleaned_state_ext_temp['state'] = cleaned_state_ext_temp['state'].str.upper()
cleaned_state_ext_temp

,state,year,extreme_temp_days
0,ALABAMA,1995,35
1,ALABAMA,1996,34
2,ALABAMA,1997,27
3,ALABAMA,1998,15
4,ALABAMA,1999,15
...,...,...,...
1243,WYOMING,2016,135
1244,WYOMING,2017,130
1245,WYOMING,2018,157
1246,WYOMING,2019,172


COMMODITIES

In [33]:
pumpkins = pd.read_csv("../bee_data/1pumpkins.csv")

In [34]:
# removing two columns (Texas & Wisconsin) that have (D) in the value column instead of a number
pumpkins.drop(pumpkins.loc[pumpkins['Value']==' (D)'].index, inplace=True)

In [35]:
pumpkins['Value'] = pumpkins['Value'].astype(float)

In [36]:
pumpkins = pumpkins[["Year", "State", "Value", "Commodity"]]

In [37]:
total_pumpkins = pumpkins.rename(columns={'Year': 'year', 'State': 'state','Commodity': 'commodity', 'Value': 'pumpkins_cwt/ac'})

In [38]:
## basically cwt/ac is * 100
total_pumpkins['lbs'] = total_pumpkins['pumpkins_cwt/ac'] * 100

In [39]:
total_pumpkins = total_pumpkins[["year", "state", "lbs", "commodity"]]
total_pumpkins

,year,state,lbs,commodity
0,2019,US TOTAL,21980.0,PUMPKINS
1,2019,CALIFORNIA,25500.0,PUMPKINS
2,2019,ILLINOIS,38500.0,PUMPKINS
3,2019,INDIANA,24500.0,PUMPKINS
4,2019,MICHIGAN,15000.0,PUMPKINS
...,...,...,...,...
174,2000,ILLINOIS,41000.0,PUMPKINS
175,2000,MICHIGAN,16000.0,PUMPKINS
176,2000,NEW YORK,20000.0,PUMPKINS
177,2000,OHIO,16500.0,PUMPKINS


In [40]:
almonds = pd.read_csv("../bee_data/2almonds.csv")

In [41]:
almonds.drop(almonds.loc[almonds['Period']=='YEAR - MAY FORECAST'].index, inplace=True)

In [42]:
almonds = almonds[["Year", "State", "Value", "Commodity"]]

In [43]:
almond_totals = almonds.rename(columns={'Year': 'year', 'State': 'state', 'Commodity': 'commodity', 'Value': 'lbs'})

In [44]:
almond_totals["lbs"] = almond_totals["lbs"].str.replace(",","")

In [45]:
almond_totals['lbs'] = almond_totals['lbs'].astype(int)
almond_totals

,year,state,lbs,commodity
0,2017,US TOTAL,2200,ALMONDS
1,2017,CALIFORNIA,2200,ALMONDS
3,2016,US TOTAL,2210,ALMONDS
4,2016,CALIFORNIA,2210,ALMONDS
5,2015,US TOTAL,2000,ALMONDS
6,2015,CALIFORNIA,2000,ALMONDS
7,2014,US TOTAL,2010,ALMONDS
8,2014,CALIFORNIA,2010,ALMONDS
9,2013,US TOTAL,2280,ALMONDS
10,2013,CALIFORNIA,2280,ALMONDS


In [46]:
apples = pd.read_csv("../bee_data/3apples.csv")

In [47]:
apples = apples[["Year", "State", "Value", "Commodity"]]

In [48]:
apple_totals = apples.rename(columns={'Year': 'year', 'State': 'state', 'Commodity': 'commodity', 'Value': 'lbs'})

In [49]:
apple_totals["lbs"] = apple_totals["lbs"].str.replace(",","")

In [50]:
apple_totals.drop(apple_totals.loc[apple_totals['lbs']==' (X)'].index, inplace=True)
apple_totals.drop(apple_totals.loc[apple_totals['lbs']==' (NA)'].index, inplace=True)
apple_totals.drop(apple_totals.loc[apple_totals['lbs']==' (D)'].index, inplace=True)

In [51]:
apple_totals['lbs'] = apple_totals['lbs'].astype(int)
apple_totals

,year,state,lbs,commodity
0,2019,US TOTAL,37500,APPLES
1,2019,CALIFORNIA,24000,APPLES
2,2019,MICHIGAN,30500,APPLES
3,2019,NEW YORK,30000,APPLES
4,2019,OREGON,30000,APPLES
...,...,...,...,...
344,2007,VERMONT,13600,APPLES
345,2007,VIRGINIA,17900,APPLES
346,2007,WASHINGTON,34000,APPLES
347,2007,WEST VIRGINIA,16000,APPLES


In [52]:
blueberries = pd.read_csv("../bee_data/4blueberries.csv")

In [53]:
blueberries.drop(blueberries.loc[blueberries['Value']==' (NA)'].index, inplace=True)

In [54]:
blueberries["Value"] = blueberries["Value"].str.replace(",","")

In [55]:
blueberries['Value'] = blueberries['Value'].astype(int)

In [56]:
total_blueberries = blueberries[["Year", "State", "Value", "Commodity"]]

In [57]:
blueberry_totals = total_blueberries.rename(columns={'Year': 'year', 'State': 'state', 'Commodity': 'commodity', 'Value': 'lbs'})
blueberry_totals

,year,state,lbs,commodity
0,2019,US TOTAL,2790,BLUEBERRIES
1,2019,MAINE,2790,BLUEBERRIES
2,2018,US TOTAL,2680,BLUEBERRIES
3,2018,MAINE,2680,BLUEBERRIES
4,2015,US TOTAL,4510,BLUEBERRIES
5,2015,MAINE,4510,BLUEBERRIES
6,2014,US TOTAL,4580,BLUEBERRIES
7,2014,MAINE,4580,BLUEBERRIES


In [58]:
cherries = pd.read_csv("../bee_data/5cherries.csv")

In [59]:
cherries.drop(cherries.loc[cherries['Value']==' (X)'].index, inplace=True)
cherries.drop(cherries.loc[cherries['Value']==' (NA)'].index, inplace=True)
cherries.drop(cherries.loc[cherries['Value']==' (D)'].index, inplace=True)

In [60]:
cherries["Value"] = cherries["Value"].str.replace(",","")

In [61]:
cherries['Value'] = cherries['Value'].astype(int)

In [62]:
total_cherries = cherries[["Year", "State", "Value", "Commodity"]]

In [63]:
cherry_total = total_cherries.rename(columns={'Year': 'year', 'State': 'state', 'Commodity': 'commodity', 'Value': 'lbs'})
cherry_total.head()

,year,state,lbs,commodity
0,2019,US TOTAL,7680,CHERRIES
1,2019,MICHIGAN,6650,CHERRIES
2,2019,NEW YORK,2700,CHERRIES
3,2019,UTAH,18000,CHERRIES
4,2019,WASHINGTON,10700,CHERRIES


In [64]:
## total_pumpkins, almond_totals, apple_totals, blueberry_totals, cherry_total
first = pd.concat([total_pumpkins, almond_totals], join="inner")
first

,year,state,lbs,commodity
0,2019,US TOTAL,21980.0,PUMPKINS
1,2019,CALIFORNIA,25500.0,PUMPKINS
2,2019,ILLINOIS,38500.0,PUMPKINS
3,2019,INDIANA,24500.0,PUMPKINS
4,2019,MICHIGAN,15000.0,PUMPKINS
...,...,...,...,...
18,2009,CALIFORNIA,1880.0,ALMONDS
19,2008,US TOTAL,2400.0,ALMONDS
20,2008,CALIFORNIA,2300.0,ALMONDS
21,2007,US TOTAL,2170.0,ALMONDS


In [65]:
## total_pumpkins, almond_totals, apple_totals, blueberry_totals, cherry_total
second = pd.concat([first, apple_totals], join="inner")
second

,year,state,lbs,commodity
0,2019,US TOTAL,21980.0,PUMPKINS
1,2019,CALIFORNIA,25500.0,PUMPKINS
2,2019,ILLINOIS,38500.0,PUMPKINS
3,2019,INDIANA,24500.0,PUMPKINS
4,2019,MICHIGAN,15000.0,PUMPKINS
...,...,...,...,...
344,2007,VERMONT,13600.0,APPLES
345,2007,VIRGINIA,17900.0,APPLES
346,2007,WASHINGTON,34000.0,APPLES
347,2007,WEST VIRGINIA,16000.0,APPLES


In [66]:
## total_pumpkins, almond_totals, apple_totals, blueberry_totals, cherry_total
third = pd.concat([second, blueberry_totals], join="inner")
third

,year,state,lbs,commodity
0,2019,US TOTAL,21980.0,PUMPKINS
1,2019,CALIFORNIA,25500.0,PUMPKINS
2,2019,ILLINOIS,38500.0,PUMPKINS
3,2019,INDIANA,24500.0,PUMPKINS
4,2019,MICHIGAN,15000.0,PUMPKINS
...,...,...,...,...
3,2018,MAINE,2680.0,BLUEBERRIES
4,2015,US TOTAL,4510.0,BLUEBERRIES
5,2015,MAINE,4510.0,BLUEBERRIES
6,2014,US TOTAL,4580.0,BLUEBERRIES


In [67]:
## total_pumpkins, almond_totals, apple_totals, blueberry_totals, cherry_total
all_crops = pd.concat([third, cherry_total], join="inner")
all_crops

,year,state,lbs,commodity
0,2019,US TOTAL,21980.0,PUMPKINS
1,2019,CALIFORNIA,25500.0,PUMPKINS
2,2019,ILLINOIS,38500.0,PUMPKINS
3,2019,INDIANA,24500.0,PUMPKINS
4,2019,MICHIGAN,15000.0,PUMPKINS
...,...,...,...,...
100,2007,OREGON,769.0,CHERRIES
102,2007,PENNSYLVANIA,5830.0,CHERRIES
103,2007,UTAH,7140.0,CHERRIES
104,2007,WASHINGTON,8850.0,CHERRIES


SQLITE TIME!!!!

In [68]:
engine = create_engine(f"sqlite:///the_hive.db")

In [69]:
cleaned_honey.to_sql(name = 'honey_prod', con = engine, if_exists='replace', index=False)

In [70]:
cleaned_colony.to_sql(name = 'colony', con = engine, if_exists='replace', index=False)

In [71]:
decline_final.to_sql(name = 'decline', con = engine, if_exists='replace', index=False)

In [72]:
cleaned_marketing_year.to_sql(name = 'marketing_year', con = engine, if_exists='replace', index=False)

In [73]:
all_crops.to_sql(name = 'commodities', con = engine, if_exists='replace', index=False)

In [74]:
cleaned_state_ext_temp.to_sql(name = 'temperature', con = engine, if_exists='replace', index=False)

In [75]:
engine.table_names()

['colony',
 'commodities',
 'decline',
 'honey_prod',
 'marketing_year',
 'temperature']